# 04 – Valores faltantes (missing values) — ASOA

En este ejercicio:
- Importo el dataset ambiental del lago.
- Exploro la estructura del DataFrame.
- Reviso valores faltantes por columna.
- Aplico dos métodos de llenado: **ffill** (propaga el último valor válido hacia adelante) y **interpolate(linear)**.
- Creo la razón **TA/DIC** y calculo medias y desviaciones por *season* y por *(season, area)*.
- Guardo los resultados a un archivo Excel.

> Nota: Para que **ffill** tenga sentido temporal, ordeno por fecha (y convierto `date` a datetime).

In [1]:
import pandas as pd

def import_csv(file):
    return pd.read_csv(file)

path = "../data/Terminos_lagoon_TA_DIC_2023_RawData.csv"
df = import_csv(path)

df.head()

,sample,date,estuary,area,station,layer_depth,season,chlorophy_microg_l,cond_microsiemens_cm,depth_m,...,do_mg_l,sal_psu,sp_cond_microsiemens_cm,turbidity_fnu,temp_c,latitude,longitude,dic_micromol_kg,ta_micromol_kg,dummy_data
0,CDL01S,5/3/2020,Candelaria,River,CDL01,Surface,Dry,0.36,7015.4,0.464,...,7.12,3.56,6547.7,1.47,28.740,18.55736,-91.25012,3915,3863,3685.0
1,CDL01F,5/3/2020,Candelaria,River,CDL01,Bottom,Dry,4.19,29886.1,7.792,...,4.90,16.97,27751.2,95.33,29.028,18.55722,-91.24990,3698,3685,NaN
2,CDL02S,5/3/2020,Candelaria,River,CDL02,Surface,Dry,0.92,16691.1,0.453,...,6.99,8.94,15429.1,5.50,29.283,18.61007,-91.24410,3724,3708,3708.0
3,CDL02F,5/3/2020,Candelaria,River,CDL02,Bottom,Dry,2.23,24847.4,1.261,...,6.52,13.87,23074.0,13.44,29.024,18.61005,-91.24403,3667,3992,3992.0
4,CDL03S,5/3/2020,Candelaria,River,CDL03,Surface,Dry,0.58,46341.6,0.465,...,6.24,28.06,43670.8,3.60,28.202,18.63166,-91.29359,2928,3023,3023.0


In [ ]:
# Explorar
print("Shape (filas, columnas):", df.shape)
print("\nColumnas:\n", df.columns.tolist())

# Convierte 'date' a datetime si existe y ordena por lógica temporal/espacial
if 'date' in df.columns:
    df['date'] = pd.to_datetime(df['date'], errors='coerce')

print("\nInfo del DataFrame:")
df.info()

print("\nEstadísticos descriptivos (numéricos):")
df.select_dtypes(include='number').describe().round(2)

Shape (filas, columnas): (106, 21)

Columnas:
 ['sample', 'date', 'estuary', 'area', 'station', 'layer_depth', 'season', 'chlorophy_microg_l', 'cond_microsiemens_cm', 'depth_m', 'do_percent_sat', 'do_mg_l', 'sal_psu', 'sp_cond_microsiemens_cm', 'turbidity_fnu', 'temp_c', 'latitude', 'longitude', 'dic_micromol_kg', 'ta_micromol_kg', 'dummy_data']

Info del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106 entries, 0 to 105
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   sample                   106 non-null    object        
 1   date                     72 non-null     datetime64[ns]
 2   estuary                  106 non-null    object        
 3   area                     106 non-null    object        
 4   station                  106 non-null    object        
 5   layer_depth              106 non-null    object        
 6   season                   106 non-

,chlorophy_microg_l,cond_microsiemens_cm,depth_m,do_percent_sat,do_mg_l,sal_psu,sp_cond_microsiemens_cm,turbidity_fnu,temp_c,latitude,longitude,dic_micromol_kg,ta_micromol_kg,dummy_data
count,106.00,106.00,106.00,106.00,106.00,106.00,106.00,106.00,106.00,106.00,106.00,106.00,106.00,99.00
mean,6.55,27895.18,1.83,89.52,6.47,17.33,27126.98,100.43,26.76,18.62,-91.60,2797.98,2912.92,2902.89
std,14.94,20931.23,2.04,29.77,2.10,13.58,20527.53,290.29,0.92,0.11,0.24,499.85,472.69,473.70
min,0.36,13.80,0.10,1.70,0.12,0.00,13.70,0.88,24.47,18.42,-91.90,2152.00,2357.00,2357.00
25%,2.55,1778.02,0.43,84.58,5.63,0.86,1706.88,5.95,26.06,18.54,-91.81,2452.25,2585.50,2561.50
50%,3.70,33202.60,0.64,97.10,6.94,19.76,31803.25,19.09,26.81,18.61,-91.79,2646.50,2823.00,2814.00
75%,5.93,47046.65,2.88,105.30,7.72,30.30,46668.98,54.76,27.15,18.70,-91.37,2963.25,3053.75,3029.00
max,150.90,59988.60,8.56,174.10,11.66,38.78,58105.00,2422.93,29.28,18.82,-91.24,4324.00,4307.00,4307.00


In [4]:
# Valores faltantes por columna
print("Valores faltantes por columna:")
df.isna().sum()


Valores faltantes por columna:


sample                      0
date                       34
estuary                     0
area                        0
station                     0
layer_depth                 0
season                      0
chlorophy_microg_l          0
cond_microsiemens_cm        0
depth_m                     0
do_percent_sat              0
do_mg_l                     0
sal_psu                     0
sp_cond_microsiemens_cm     0
turbidity_fnu               0
temp_c                      0
latitude                    0
longitude                   0
dic_micromol_kg             0
ta_micromol_kg              0
dummy_data                  7
dtype: int64

## Llenado de faltantes con ffill / bfill

- **ffill**: propaga el último valor válido hacia adelante.  
- **bfill**: usa el siguiente valor válido hacia atrás (no lo aplico aquí, pero es equivalente con `method="bfill"`).

> Para que ffill sea coherente, ordeno primero por `estuary`, `area`, `station` y `date` (si existen).


In [6]:
orden_cols = [c for c in ['estuary','area','station','date'] if c in df.columns]
df_ffill = df.copy()

if orden_cols:
    df_ffill = df_ffill.sort_values(orden_cols)

df_ffill = df_ffill.fillna(method="ffill")

print("Faltantes después de ffill:")
df_ffill.isna().sum()


Faltantes después de ffill:


/var/folders/1q/0d1wc4291_z5gfjdgg375t4m0000gn/T/ipykernel_13529/3425277351.py:7: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_ffill = df_ffill.fillna(method="ffill")


sample                     0
date                       0
estuary                    0
area                       0
station                    0
layer_depth                0
season                     0
chlorophy_microg_l         0
cond_microsiemens_cm       0
depth_m                    0
do_percent_sat             0
do_mg_l                    0
sal_psu                    0
sp_cond_microsiemens_cm    0
turbidity_fnu              0
temp_c                     0
latitude                   0
longitude                  0
dic_micromol_kg            0
ta_micromol_kg             0
dummy_data                 0
dtype: int64

## Interpolación lineal

- **interpolate(method="linear")**: interpola numéricos de forma lineal.
- No afecta columnas no numéricas.


In [7]:
df_interp = df.copy()
df_interp = df_interp.interpolate(method='linear', limit_direction='both')

print("Faltantes después de interpolate(linear):")
df_interp.isna().sum()


Faltantes después de interpolate(linear):


/var/folders/1q/0d1wc4291_z5gfjdgg375t4m0000gn/T/ipykernel_13529/1477750768.py:2: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_interp = df_interp.interpolate(method='linear', limit_direction='both')


sample                     0
date                       0
estuary                    0
area                       0
station                    0
layer_depth                0
season                     0
chlorophy_microg_l         0
cond_microsiemens_cm       0
depth_m                    0
do_percent_sat             0
do_mg_l                    0
sal_psu                    0
sp_cond_microsiemens_cm    0
turbidity_fnu              0
temp_c                     0
latitude                   0
longitude                  0
dic_micromol_kg            0
ta_micromol_kg             0
dummy_data                 0
dtype: int64

## Ejercicios

1. Crear una nueva columna `TA_DIC_ratio = ta_micromol_kg / dic_micromol_kg`.  
2. Calcular media y desviación de `TA_DIC_ratio` por `season`.  
3. Calcular media y desviación por `season` y `area`.  
4. Guardar los resultados a un Excel `TA_DIC_Season_Areas.xlsx`.


In [8]:
# Trabajo con la versión interpolada (df_interp) para minimizar NaNs en el ratio
work = df_interp.copy()

# Crear TA_DIC_ratio de manera segura (evita divisiones por cero o nulos)
num = work.get('ta_micromol_kg')
den = work.get('dic_micromol_kg')

if num is None or den is None:
    raise ValueError("Faltan columnas 'ta_micromol_kg' y/o 'dic_micromol_kg' en el dataset.")

# Evita división por cero/nulo
work['TA_DIC_ratio'] = num / den.replace({0: pd.NA})

# Resúmenes por season
if 'season' in work.columns:
    ratio_by_season = work.groupby('season')['TA_DIC_ratio'].agg(['mean','std','count']).round(3)
else:
    ratio_by_season = pd.DataFrame({'mean':[], 'std':[], 'count':[]})

# Resúmenes por season y area
if 'season' in work.columns and 'area' in work.columns:
    ratio_by_season_area = (
        work.groupby(['season','area'])['TA_DIC_ratio']
            .agg(['mean','std','count']).round(3)
    )
else:
    ratio_by_season_area = pd.DataFrame({'mean':[], 'std':[], 'count':[]})

ratio_by_season, ratio_by_season_area


(         mean    std  count
 season                     
 Dry     1.059  0.086     72
 Rainy   1.022  0.101     34,
                mean    std  count
 season area                      
 Dry    Coast  1.116  0.115     24
        Plume  1.060  0.036     24
        River  1.000  0.038     24
 Rainy  Coast  1.090  0.130     12
        Plume  1.026  0.049     11
        River  0.945  0.025     11)

## Guardar resultados a Excel

Se guarda a `tareas/TA_DIC_Season_Areas.xlsx` con dos hojas:
- `by_season`
- `by_season_area`

Si no está disponible el motor de Excel, se guarda un CSV como alternativa.


In [10]:
# Guardar a Excel (o CSV de respaldo)
out_xlsx = "TA_DIC_Season_Areas.xlsx"  # se guardará en la misma carpeta del notebook: 'tareas/'
try:
    with pd.ExcelWriter(out_xlsx, engine="openpyxl") as writer:
        ratio_by_season.to_excel(writer, sheet_name="by_season")
        ratio_by_season_area.to_excel(writer, sheet_name="by_season_area")
    print(f"Archivo Excel guardado: {out_xlsx}")
except Exception as e:
    print("No se pudo escribir Excel (quizá falta 'openpyxl'). Guardo CSVs como respaldo.")
    ratio_by_season.to_csv("TA_DIC_Season.csv")
    ratio_by_season_area.to_csv("TA_DIC_Season_Areas.csv")
    print("Archivos guardados: TA_DIC_Season.csv, TA_DIC_Season_Areas.csv")


Archivo Excel guardado: TA_DIC_Season_Areas.xlsx
